In [1]:
import graphlab
import numpy as np
import pandas as pd

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.7.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
best_answer = pd.read_csv('./DIVIDE/MATCH/best_answer.csv')
second_answer = pd.read_csv('./DIVIDE/MATCH/second_answer.csv')
third_answer = pd.read_csv('./DIVIDE/MATCH/third_answer.csv')

In [3]:
init_qid = best_answer['question_id']
init_qc = best_answer['question_content']
init_qt = best_answer['question_tags']

In [4]:
# best_answer    # 1959 rows × 18 columns

# Use KNN Model Find Similar Question Content

In [5]:
question_content = best_answer[['question_id','question_content','question_tags']]   # 1959 rows × 3 columns

In [6]:
# question_content
question_content.to_csv('./DIVIDE/SIMILAR/Question_Content.csv', encoding='utf-8', index = False)

In [7]:
# Use graphlab to read question_content
question_content = graphlab.SFrame.read_csv('./DIVIDE/SIMILAR/Question_Content.csv')

[INFO] 1461064865 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /home/zpgao/anaconda2/lib/python2.7/site-packages/certifi/cacert.pem
1461064865 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to beyond_acm@163.com and will expire on January 04, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-7861 - Server binary: /home/zpgao/anaconda2/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1461064865.log
[INFO] GraphLab Server Version: 1.7.1


PROGRESS: Finished parsing file /home/zpgao/ML/Best_Answer/Feature_Extraction/DIVIDE/SIMILAR/Question_Content.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.021802 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /home/zpgao/ML/Best_Answer/Feature_Extraction/DIVIDE/SIMILAR/Question_Content.csv
PROGRESS: Parsing completed. Parsed 1959 lines in 0.009021 secs.


In [8]:
# question_content

In [9]:
# add 'text' && 'word_count' && 'tfidf' features
question_content['text'] = question_content['question_tags'] +  question_content['question_content'] 
question_content['word_count'] = graphlab.text_analytics.count_words(question_content['text'])
tfidf = graphlab.text_analytics.tf_idf(question_content['word_count'])
question_content['tfidf'] = tfidf

In [10]:
knn_model = graphlab.nearest_neighbors.create(question_content, features=['tfidf'], label='question_id')

PROGRESS: Starting brute force nearest neighbors model training.


In [11]:
# question_content
# len( question_content ) 1959

In [12]:
# q0 = question_content[0:1]
# Similar_Qid_List is a dict 
# key : Index From 0 - 1958 
# value : Similar Qid List [Qid1, Qid2, Qid3, Qid4, Qid5]
# question_content[Index]['qustion_id'] = Qid1
Similar_Qid_List = {}
for i in range( len(question_content) ) :
    key = 'Q' + str(i) + '_Similar_List'
    Q_i = question_content[i:i+1]
    value = knn_model.query(Q_i, k=5)['reference_label']
    Similar_Qid_List[key] = value

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0510465   | 1.961ms      |
PROGRESS: | Done         |         | 100         | 4.367ms      |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0510465   | 1.053ms      |
PROGRESS: | Done         |         | 100         | 5.887ms      |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | #

In [13]:
# Similar_Qid_List['Q0_Similar_List'][0]

In [14]:
Original_List = []
Similar1_List = []
Similar2_List = []
Similar3_List = []

for i in range( len(question_content) ) :
    key = 'Q' + str(i) + '_Similar_List'
    value = Similar_Qid_List[key][0]
    Original_List.append(value)
    
for i in range( len(question_content) ) :
    key = 'Q' + str(i) + '_Similar_List'
    value = Similar_Qid_List[key][1]
    Similar1_List.append(value)
    
for i in range( len(question_content) ) :
    key = 'Q' + str(i) + '_Similar_List'
    value = Similar_Qid_List[key][2]
    Similar2_List.append(value)
    
for i in range( len(question_content) ) :
    key = 'Q' + str(i) + '_Similar_List'
    value = Similar_Qid_List[key][3]
    Similar3_List.append(value)

In [15]:
# len( Original_List )  # 1959
# len( Similar1_List )  # 1959
# len( Similar2_List )  # 1959
# len( Similar3_List )  # 1959
# len( set(Similar1_List) )  # 1048
# len( set(Similar2_List) )  # 1044 
# len( set(Similar3_List) )  # 1062

In [16]:
def Get_Similar_DataSet( DataSet, Similar_List ) :
    # Generate
    Similar_DataSet = DataSet[ DataSet['question_id'] == Similar_List[0] ] 
    for i in range(1, len(Similar_List)) :
        Similar_DataSet = Similar_DataSet.append( DataSet[ DataSet['question_id'] == Similar_List[i] ] )
    # Reset index
    Similar_DataSet = Similar_DataSet.reset_index()
    Similar_DataSet = Similar_DataSet.drop('index', axis=1)
    # Replace Question_id / Question_content / Question_tags
    Similar_DataSet['question_id'] = init_qid
    Similar_DataSet['question_content'] = init_qc
    Similar_DataSet['question_tags'] = init_qt
    
    return Similar_DataSet    

In [17]:
Similar_best01 = Get_Similar_DataSet( best_answer, Similar1_List )
Similar_second01 = Get_Similar_DataSet( second_answer, Similar1_List )
Similar_third01 = Get_Similar_DataSet( third_answer, Similar1_List )

Similar_best02 = Get_Similar_DataSet( best_answer, Similar2_List )
Similar_second02 = Get_Similar_DataSet( second_answer, Similar2_List)
Similar_third02 = Get_Similar_DataSet( third_answer, Similar2_List) 

Similar_best03 = Get_Similar_DataSet( best_answer, Similar3_List )
Similar_second03 = Get_Similar_DataSet( second_answer, Similar3_List )
Similar_third03 = Get_Similar_DataSet( third_answer, Similar3_List )

In [19]:
Similar_best01.to_csv( './DIVIDE/SIMILAR/best_similar_01.csv', encoding='utf-8', index = False)
Similar_second01.to_csv( './DIVIDE/SIMILAR/second_similar_01.csv', encoding='utf-8', index = False)
Similar_third01.to_csv( './DIVIDE/SIMILAR/third_similar_01.csv', encoding='utf-8', index = False)

Similar_best02.to_csv( './DIVIDE/SIMILAR/best_similar_02.csv', encoding='utf-8', index = False)
Similar_second02.to_csv( './DIVIDE/SIMILAR/second_similar_02.csv', encoding='utf-8', index = False)
Similar_third02.to_csv( './DIVIDE/SIMILAR/third_similar_02.csv', encoding='utf-8', index = False)

Similar_best03.to_csv( './DIVIDE/SIMILAR/best_similar_03.csv', encoding='utf-8', index = False)
Similar_second03.to_csv( './DIVIDE/SIMILAR/second_similar_03.csv', encoding='utf-8', index = False)
Similar_third03.to_csv( './DIVIDE/SIMILAR/third_similar_03.csv', encoding='utf-8', index = False)